In [1]:
import pandas as pan;
import numpy as np;
from datetime import datetime;
import math as m;

In [2]:
matrix_data = pan.read_csv("hotel_booking_data.csv");

In [3]:
print(matrix_data.head(5));
print(matrix_data.columns)

          hotel  is_canceled  lead_time  arrival_date_year arrival_date_month   
0  Resort Hotel            0        342               2015               July  \
1  Resort Hotel            0        737               2015               July   
2  Resort Hotel            0          7               2015               July   
3  Resort Hotel            0         13               2015               July   
4  Resort Hotel            0         14               2015               July   

   arrival_date_week_number  arrival_date_day_of_month   
0                        27                          1  \
1                        27                          1   
2                        27                          1   
3                        27                          1   
4                        27                          1   

   stays_in_weekend_nights  stays_in_week_nights  adults  ...  customer_type   
0                        0                     0       2  ...      Transient  \
1   

In [4]:
pan.to_datetime(arg = matrix_data["reservation_status_date"]);

In [5]:
#number of rows and whether any empty data exists
print(matrix_data.tail(1).index[0]);
#columns with the most empty data
most_missing_info = matrix_data.isna().sum().idxmax();

119389


In [6]:
#add another column to show the comprehensive arrival date, and drop the useless one in the data 
#define a concat function then vectorize it 
def concatdate(year: int, month: str, day: int) -> str:
    return str(year) + "-" + month + "-" + str(day);
matrix_data1 = matrix_data;
matrix_data1["arrival_date"] = np.vectorize(concatdate)(matrix_data["arrival_date_year"], matrix_data["arrival_date_month"], matrix_data["arrival_date_day_of_month"]);
matrix_data1 = matrix_data1.drop(["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"], axis = 1);
matrix_data1["arrival_date"] = pan.to_datetime(arg = matrix_data1["arrival_date"]);

In [7]:
#drop columns that are basically useless, fill those that may be inferred
matrix_data1 = matrix_data1.drop(most_missing_info, axis = 1);

In [8]:
#most commonly used country codes 
print(matrix_data["country"].value_counts()[: 5]);

country
PRT    48590
GBR    12129
FRA    10415
ESP     8568
DEU     7287
Name: count, dtype: int64


In [9]:
#person with the most ADR
print(matrix_data1['name'].iloc[matrix_data1["adr"].idxmax()]);
#the mean ADR
print(matrix_data1["adr"].mean());
print(matrix_data1['stays_in_week_nights'].mean() + matrix_data1["stays_in_weekend_nights"].mean());


Daniel Walter
101.83112153446686
3.4279001591423066


In [10]:
#concatenate the stay columns into one column
def add(a: int, b: int ):
     return a+b;
matrix_data1["total_stay_days"] = np.vectorize(add)(matrix_data1["stays_in_week_nights"], matrix_data1["stays_in_weekend_nights"])
matrix_data1.drop(["stays_in_week_nights"], inplace=True, axis = 1);
matrix_data1.drop(["stays_in_weekend_nights"], inplace=True, axis = 1);
print(matrix_data1.columns);

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_week_number',
       'adults', 'children', 'babies', 'meal', 'country', 'market_segment',
       'distribution_channel', 'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'name', 'email',
       'phone-number', 'credit_card', 'arrival_date', 'total_stay_days'],
      dtype='object')


In [11]:
print(matrix_data1["total_stay_days"].describe());
#note how the mean of this series is equal to the cumulative mean gotten when we added the redundant columns total weekday and total weeknight stays

count    119390.000000
mean          3.427900
std           2.557439
min           0.000000
25%           2.000000
50%           3.000000
75%           4.000000
max          69.000000
Name: total_stay_days, dtype: float64


In [12]:
#average total cost for a stay
print(np.vectorize(lambda a, b: a* b)(matrix_data1["adr"], matrix_data1["total_stay_days"]).mean());

357.84820780634897


In [13]:
#made exactly five requests 
print(matrix_data1[matrix_data1["total_of_special_requests"] == 5][["name", "email"]]);

                          name                          email
7860             Amanda Harper           Amanda.H66@yahoo.com
11125            Laura Sanders      Sanders_Laura@hotmail.com
14596              Tommy Ortiz            Tommy_O@hotmail.com
14921           Gilbert Miller         Miller.Gilbert@aol.com
14922           Timothy Torres         TTorres@protonmail.com
24630          Jennifer Weaver             Jennifer_W@aol.com
27288           Crystal Horton             Crystal.H@mail.com
27477           Brittney Burke       Burke_Brittney16@att.com
29906          Cynthia Cabrera    Cabrera.Cynthia@xfinity.com
29949              Sarah Floyd              Sarah_F@gmail.com
32267           Michelle Villa         Michelle.Villa@aol.com
39027           Nichole Hebert       Hebert.Nichole@gmail.com
39129         Lindsey Mckenzie       Lindsey.Mckenzie@att.com
39525           Ashley Edwards       Edwards.Ashley@yahoo.com
70114       Christopher Torres   Torres.Christopher@gmail.com
78819   

In [14]:
#percentage are repeat guests
# print(matrix_data1["is_repeated_guest"].head());
print(len(matrix_data1[matrix_data1["is_repeated_guest"] == 1])/len(matrix_data1) * 100);

3.191222045397437


In [15]:
#top 5 most common last names in the dataset
ser = np.vectorize(lambda a: a.split()[-1])(matrix_data1["name"]);
print(pan.Series(ser).value_counts()[: 5]);

Smith       2503
Johnson     1990
Williams    1618
Jones       1434
Brown       1423
Name: count, dtype: int64


In [17]:
#table that counts what day of the week customers arrived
#no need to map, simply create a new column that takes the datetime object I've already collected, assign it to a column, then print the value_counts
print(matrix_data1["arrival_date"].map(lambda x: x.strftime("%A")).value_counts());
#okay apparently there was a need to map
#I have to say though, I am elegant

arrival_date
Friday       19631
Thursday     19254
Monday       18171
Saturday     18055
Wednesday    16139
Sunday       14141
Tuesday      13999
Name: count, dtype: int64
